In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive')

In [3]:
import string
import numpy as np
import pandas as pd
from numpy import array
from PIL import Image
import pickle
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import LambdaLR

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from model import TransformerModel
from torch.utils.data import DataLoader
import torch.optim as optim
from torch import nn
import torch
from dataset import DataGenerator
import time
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler


from utils import *


tf.get_logger().setLevel('ERROR')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


In [4]:
captions_path = os.path.join('/content/drive/MyDrive/data/captions.txt')

In [5]:
top_k = 5000
caption_dict, word_to_index, index_to_word = preprocess_captions(captions_path)


In [6]:
keys = list(caption_dict.keys())
train_keys, test_keys = train_test_split(keys, test_size = 0.2, random_state = 42)
train_keys, val_keys = train_test_split(train_keys, test_size = 0.25, random_state = 42)

In [7]:
features_path = os.path.join('/content/drive/MyDrive/encodings_swin.pkl')
with open(features_path, 'rb') as f:
    features_dict = pickle.load(f)

In [8]:
train_captions = {k: caption_dict[k] for k in train_keys}
val_captions = {k: caption_dict[k] for k in val_keys}
test_captions = {k: caption_dict[k] for k in test_keys}

train_features = {k: features_dict[k] for k in train_keys}
val_features = {k: features_dict[k] for k in val_keys}
test_features = {k: features_dict[k] for k in test_keys}

In [9]:
tokenizer = Tokenizer(num_words = top_k,
                      oov_token="<unk>",
                      filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

In [10]:
tc = get_captions(train_captions)
tokenizer.fit_on_texts(tc)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
max_seq_length = get_max_length(caption_dict)

In [11]:
train_data_generator = DataGenerator(train_captions, train_features, tokenizer, max_seq_length)
train_data = DataLoader(train_data_generator, batch_size=32)
val_data_generator = DataGenerator(val_captions, val_features, tokenizer, max_seq_length)
val_data = DataLoader(val_data_generator, batch_size=32)
test_data_generator = DataGenerator(test_captions, test_features, tokenizer, max_seq_length)
test_data = DataLoader(test_data_generator, batch_size=32)

In [12]:
model = TransformerModel(model_dim = 512, input_dim = 768, num_heads = 8, num_layers = 4, vocab_size = top_k + 1, sequence_length = top_k + 1, height = 7, width = 7)

In [13]:
model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive/model.pth')))

<All keys matched successfully>

In [15]:
class LRScheduler(_LRScheduler):
    def __init__(self,
                 optimizer: Optimizer,
                 dim_embed: int,
                 warmup_steps: int,
                 last_epoch: int=-1,
                 verbose: bool=False) -> None:

        self.dim_embed = dim_embed
        self.warmup_steps = warmup_steps
        self.num_param_groups = len(optimizer.param_groups)

        super().__init__(optimizer, last_epoch, verbose)

    def get_lr(self) -> float:
        lr = calc_lr(self._step_count, self.dim_embed, self.warmup_steps)
        return [lr] * self.num_param_groups


def calc_lr(step, dim_embed, warmup_steps):
    return dim_embed**(-0.5) * min(step**(-0.5), step * warmup_steps**(-1.5))

In [16]:
loss_fn = nn.CrossEntropyLoss(reduction = 'none')
optimizer = optim.Adam(model.parameters(), lr = 0.001, betas = (0.9, 0.98), eps = 1e-9)
lr_mul = 1.0 / 512 ** 0.5
scheduler = LRScheduler(optimizer, dim_embed = 512, warmup_steps = 4000)


In [17]:
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [18]:
def loss_function(real, pred):
    mask = torch.logical_not(torch.eq(real, 0))
    loss_ = loss_fn(pred.reshape(-1, pred.size(-1)), real.reshape(-1))

    mask = mask.float()
    loss_ *= mask.view_as(loss_)

    return torch.sum(loss_) / torch.sum(mask)

In [19]:
def evaluate_step(img_tensor, tar):
    target_in = tar[:, :-1]
    target_real = tar[:, 1:]

    predictions = model(img_tensor, target_in)
    loss = loss_function(target_real, predictions)

    _, predicted_ids = torch.max(predictions, dim=-1)
    accuracy = torch.sum(predicted_ids == target_real).item() / target_real.numel()

    return loss, accuracy

In [ ]:
for epoch in range(num_epochs):
    start = time.time()

    model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    with tqdm(total=len(train_data), desc=f'Epoch {epoch + 1} Training') as pbar:
        for batch, (img_tensor, tar) in enumerate(train_data):
            img_tensor, tar = img_tensor.to(device), tar.to(device)
            img_tensor = img_tensor.float()
            tar = tar.long()
            optimizer.zero_grad()
            loss, batch_accuracy = evaluate_step(img_tensor, tar)
            batch_loss = loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_loss += batch_loss
            epoch_accuracy += batch_accuracy

            pbar.set_postfix({'Loss': batch_loss, 'Accuracy': batch_accuracy})
            pbar.update()

    mean_epoch_loss = epoch_loss / len(train_data)
    mean_epoch_accuracy = epoch_accuracy / len(train_data)

    print('Epoch {} Training Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, mean_epoch_loss, mean_epoch_accuracy))

    if (epoch + 1) % 5 == 0:
        torch.save(model.state_dict(), 'model.pth')

    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0

    with torch.no_grad():
        for batch, (val_img_tensor, val_tar) in enumerate(val_data):
            val_img_tensor, val_tar = val_img_tensor.to(device), val_tar.to(device)
            val_img_tensor = val_img_tensor.float()
            val_tar = val_tar.long()
            val_batch_loss, val_batch_accuracy = evaluate_step(val_img_tensor, val_tar)

            val_loss += val_batch_loss
            val_accuracy += val_batch_accuracy

            pbar.set_postfix({'Loss': val_batch_loss, 'Accuracy': val_batch_accuracy})
            pbar.update()

    mean_val_loss = val_loss / len(val_data)
    mean_val_accuracy = val_accuracy / len(val_data)

    print('Epoch {} Validation Loss {:.4f} Validation Accuracy {:.4f}'.format(epoch + 1, mean_val_loss, mean_val_accuracy))

Epoch 1 Training: 100%|██████████| 152/152 [00:14<00:00, 10.44it/s, Loss=6.29, Accuracy=0.0344]


Epoch 1 Training Loss 7.2670 Accuracy 0.0265
Epoch 1 Validation Loss 6.1868 Validation Accuracy 0.0379


Epoch 2 Training: 100%|██████████| 152/152 [00:14<00:00, 10.34it/s, Loss=5.1, Accuracy=0.0675]


Epoch 2 Training Loss 5.6075 Accuracy 0.0539
Epoch 2 Validation Loss 4.9716 Validation Accuracy 0.0666


Epoch 3 Training: 100%|██████████| 152/152 [00:16<00:00,  9.43it/s, Loss=4.32, Accuracy=0.0799]


...

Epoch 98 Training Loss 0.8363 Accuracy 0.2313
Epoch 98 Validation Loss 4.7661 Validation Accuracy 0.0956


Epoch 99 Training: 100%|██████████| 152/152 [00:14<00:00, 10.68it/s, Loss=0.623, Accuracy=0.238]


Epoch 99 Training Loss 0.8202 Accuracy 0.2336
Epoch 99 Validation Loss 4.7362 Validation Accuracy 0.0972


Epoch 100 Training: 100%|██████████| 152/152 [00:14<00:00, 10.66it/s, Loss=0.649, Accuracy=0.247]


Epoch 100 Training Loss 0.8069 Accuracy 0.2339
Epoch 100 Validation Loss 4.7815 Validation Accuracy 0.0913


In [26]:
def beam_search(model, img_tensor, start, end, k=3, max_len=50):
    batch_size = img_tensor.size(0)
    sequences_batch = [[[(start,), 1.0]] for _ in range(batch_size)]

    for _ in range(max_len):
        all_candidates_batch = [[] for _ in range(batch_size)]
        all_sequences_complete = True

        for batch in range(batch_size):
            sequences = sequences_batch[batch]
            all_candidates = all_candidates_batch[batch]
            for seq_and_score in sequences:
                seq, score = seq_and_score[0], seq_and_score[1]
                if len(seq) >= max_len or seq[-1] == end:
                    all_candidates.append(seq_and_score)
                    continue

                all_sequences_complete = False

                dec_input = torch.tensor([seq], dtype=torch.long, device=device)

                predictions = model(img_tensor[batch].unsqueeze(0), dec_input)
                predictions = predictions[:, -1:, :]
                predictions = torch.softmax(predictions, dim=-1)

                top_k_probs, top_k_tokens = torch.topk(predictions, k)
                top_k_log_probs = top_k_probs.log().squeeze()

                for j in range(k):
                    next_token = top_k_tokens[0][0][j].item()
                    if next_token == end:
                        all_candidates.append((seq + (next_token,), score + top_k_log_probs[j].item()))
                        continue
                    else:
                        candidate = (seq + (next_token,), score + top_k_log_probs[j].item())
                        all_candidates.append(candidate)

            ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)

            sequences_batch[batch] = ordered[:k]

        if all_sequences_complete:
            break

    return sequences_batch


In [27]:
def evaluate_bleu(references, hypothesis, weights):
    bleu_scores = []
    for ref in references:
        ref_tokens = [token for token in ref.split()[1:-1] if token != '<unk>']
        hyp_tokens = [token for token in hypothesis.split()[1:-1] if token != '<unk>']
        bleu_score = sentence_bleu([ref_tokens], hyp_tokens, weights=weights, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu_score * 100)
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu_score



weights_list = [(1.0, 0, 0, 0), (0.5, 0.5, 0, 0), (0.3, 0.3, 0.3, 0), (0.25, 0.25, 0.25, 0.25)]


total_bleu_scores = {weights: [] for weights in weights_list}

for batch, (img_tensor, tar) in enumerate(test_data):
    img_tensor, tar = img_tensor.to(device), tar.to(device)
    img_tensor = img_tensor.float()
    tar = tar.long()
    start = tokenizer.word_index['<start>']
    end = tokenizer.word_index['<end>']

    batch_bleu_scores = {weights: [] for weights in weights_list}

    output_sequences_batch = beam_search(model, img_tensor, start, end, k=3, max_len=50)

    for i, (output_sequences, target) in enumerate(zip(output_sequences_batch, tar)):
        target_sentence = tokenizer.sequences_to_texts([target.cpu().numpy()])[0]

        best_generated_sentences = {weights: None for weights in weights_list}
        best_scores = {weights: -1 for weights in weights_list}

        for j, (seq, _) in enumerate(output_sequences):
            generated_sentence = tokenizer.sequences_to_texts([seq])[0]

            for weights in weights_list:
                bleu_score = evaluate_bleu([target_sentence], generated_sentence, weights)

                if bleu_score > best_scores[weights]:
                    best_generated_sentences[weights] = generated_sentence
                    best_scores[weights] = bleu_score

        for weights in weights_list:
            batch_bleu_scores[weights].append(best_scores[weights])

    for weights in weights_list:
        avg_batch_bleu_score = np.mean(batch_bleu_scores[weights])
        total_bleu_scores[weights].append(avg_batch_bleu_score)
    break

overall_avg_bleu_scores = {weights: np.mean(scores) for weights, scores in total_bleu_scores.items()}

for weights, avg_score in overall_avg_bleu_scores.items():
    print(f"Average BLEU Score for weights {weights}: {avg_score}")



Average BLEU Score for weights (1.0, 0, 0, 0): 3.8530133027160964
Average BLEU Score for weights (0.5, 0.5, 0, 0): 2.050459568302278
Average BLEU Score for weights (0.3, 0.3, 0.3, 0): 1.6340909479885535
Average BLEU Score for weights (0.25, 0.25, 0.25, 0.25): 0.9452908998030194


In [28]:
def evaluate(img_tensor, target):
    start_token = tokenizer.word_index['<start>']
    end_token = tokenizer.word_index['<end>']

    beam_width = 3
    beam_result = beam_search(model, img_tensor, start_token, end_token, k=beam_width)

    output_sequence = beam_result[0][0][0][1:-1]
    output_sequence = [tokenizer.index_word[idx] for idx in output_sequence]

    target_seq = [tokenizer.index_word[int(idx)] for idx in target if idx not in [1, 2, 0, 3]]

    print('Target sequence: ', ' '.join(target_seq))
    print('Predicted sequence: ', ' '.join(output_sequence))

    bleu_score = sentence_bleu([target_seq], output_sequence,
                               weights=(0.25, 0.25, 0.25, 0.25),
                               smoothing_function=SmoothingFunction().method1) * 100

    print('Bleu Score:', bleu_score)


In [29]:
i = 0
for img_tensors, targets in test_data:
  target = targets[i]
  img_tensor = img_tensors[i]
  img_tensor = img_tensor[None, :, :]
  img_tensor, target = img_tensor.to(device), target.to(device)
  evaluate(img_tensor, target)
  i += 1
  if (i == 5):
    break

Target sequence:  black and white dog is jumping through field of brown grass
Predicted sequence:  black and white dog running through field
Bleu Score: 27.610369103579487
Target sequence:  little girl in bathing suit leaps up in the water
Predicted sequence:  young boy jumping into the ocean
Bleu Score: 2.096016611399374
Target sequence:  the two children watch the small yellow dog on the tree
Predicted sequence:  four dogs are playing around tree
Bleu Score: 1.774239756616722
Target sequence:  boy jumping rail on his skateboard
Predicted sequence:  young man wearing gray shirt and jeans skateboards down street
Bleu Score: 0
Target sequence:  two people on grassy plain are gathering parachute that one of the people just used
Predicted sequence:  two people are in the air above the grass
Bleu Score: 3.096277640217347
